# Исследование надежности заемщиков

**Цель исследования** - определить степень влияния семейного положения и количества детей заёмщика на факт своевременного погашения кредита. 

Для достижения поставленной цели необходимо решить следующие задачи:
1. [Изучение и обработка данных](#1)
2. [Проверка данных на аномалии и исправления](#2)
3. [Изменение типов данных](#3)
4. [Удаление дубликатов](#4)
5. [Формирование дополнительных датафреймов словарей, декомпозиция исходного датафрейма](#5)
6. [Категоризация дохода](#6)
7. [Категоризация целей кредита](#7)
8. [Определения зависимостей](#8)
9. [Общий вывод](#9)

<a id='1'></a>
### 1. Изучение и обработка данных

In [3]:
import pandas as pd
data = pd.read_csv('/datasets/data.csv')
display(data.head())
data.info()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


Данные содержат 21525 строк и 12 колонок:
 - children - количество детей в семье;
 - days_employed - общий трудовой стаж, в днях;
 - dob_years - возраст клиента;           
 - education - уровень образования клиента;           
 - education_id - идентификатор уровня образования;     
 - family_status - семейное положение;      
 - family_status_id - идентификатор семейного положения;   
 - gender - пол клиента;   
 - income_type - тип занятости клиента;
 - debt - имел ли задолженности по кредитам (0 - не имел, 1 - имел);                
 - total_income - ежемесячных доход;        
 - purpose - цель получения кредита.  
 
Количество значений в колонках days_employed и total_income различаются - значит есть пропущенные значения.

In [4]:
#Заполнение пропусков
data[data['days_employed'].isna()]
data[data['total_income'].isna()]

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
12,0,NaN,65,среднее,1,гражданский брак,1,M,пенсионер,0,NaN,сыграть свадьбу
26,0,NaN,41,среднее,1,женат / замужем,0,M,госслужащий,0,NaN,образование
29,0,NaN,63,среднее,1,Не женат / не замужем,4,F,пенсионер,0,NaN,строительство жилой недвижимости
41,0,NaN,50,среднее,1,женат / замужем,0,F,госслужащий,0,NaN,сделка с подержанным автомобилем
55,0,NaN,54,среднее,1,гражданский брак,1,F,пенсионер,1,NaN,сыграть свадьбу
...,...,...,...,...,...,...,...,...,...,...,...,...
21489,2,NaN,47,Среднее,1,женат / замужем,0,M,компаньон,0,NaN,сделка с автомобилем
21495,1,NaN,50,среднее,1,гражданский брак,1,F,сотрудник,0,NaN,свадьба
21497,0,NaN,48,ВЫСШЕЕ,0,женат / замужем,0,F,компаньон,0,NaN,строительство недвижимости
21502,1,NaN,42,среднее,1,женат / замужем,0,F,сотрудник,0,NaN,строительство жилой недвижимости


Пропущенные значения по столбцам days_employed и total_income находятся в одних и тех же строках, ячейки заполнены NaN и составляют 10% от общего числа ячеек.
Наличие пропусков может быть связано с непредоставлением клиентом (заёмщиком) информации о доходах.


In [5]:
data['days_employed'] = data['days_employed'].abs()
data['days_employed'] = data['days_employed'].fillna(data['days_employed'].median())
data['total_income'] = data['total_income'].fillna(data['total_income'].median())
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     21525 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      21525 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


Пропущенные значения в колонках days_employed и total_income заменены на медианное значение по столбцу, как характеризующее состояние выборки в количественном значении.

<a id='2'></a>
### 2. Проверка данных на аномалии и исправления

#### Колонка children - количество детей заёмщика

In [6]:
data['children'].sort_values().unique()

array([-1,  0,  1,  2,  3,  4,  5, 20], dtype=int64)

In [7]:
data.loc[data['children'] == -1]['children'].count()

47

In [8]:
data.loc[data['children'] == 20]['children'].count()

76

При проверке значений по колонке children выявлены аномальные значения: -1 (47 строк) и 20 (76 строк).
Возможно данные аномальные значения возникли в результате опечатки. Заменим -1 на 1, 20 на 2.

In [9]:
data['children'] = data['children'].replace(-1, 1)
data['children'] = data['children'].replace(20, 2)
data['children'].sort_values().unique()

array([0, 1, 2, 3, 4, 5], dtype=int64)

#### Колонка dob_years - возраст заёмщика

In [10]:
data['dob_years'].sort_values().unique()

array([ 0, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34,
       35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51,
       52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68,
       69, 70, 71, 72, 73, 74, 75], dtype=int64)

In [11]:
data.loc[data['dob_years'] == 0]['dob_years'].count()

101

При проверке значений по колонке dob_years выявлены аномальные значения: 0 лет (101 строка). Заменим на медианное значение.

In [12]:
data['dob_years'] = data['dob_years'].replace(0, data['dob_years'].median())
data['dob_years'].sort_values().unique()

array([19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35,
       36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52,
       53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69,
       70, 71, 72, 73, 74, 75], dtype=int64)

#### Колонка family_status - семейное положение и family_status_id - идентификатор семейного положения

In [13]:
data['family_status'].sort_values().unique()

array(['Не женат / не замужем', 'в разводе', 'вдовец / вдова',
       'гражданский брак', 'женат / замужем'], dtype=object)

In [14]:
data['family_status_id'].sort_values().unique()

array([0, 1, 2, 3, 4], dtype=int64)

Без аномальных значений.

#### Колонка gender - пол заёмщика

In [15]:
data['gender'].sort_values().unique()

array(['F', 'M', 'XNA'], dtype=object)

In [16]:
data.loc[data['gender'] == 'XNA']['gender'].count()

1

In [17]:
data[data['gender'] == 'XNA']

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
10701,0,2358.600502,24,неоконченное высшее,2,гражданский брак,1,XNA,компаньон,0,203905.157261,покупка недвижимости


В Российской Федерации официально признано два пола: мужской и женский. Вероятно значение XNA - опечатка. Для достижения цели исследования данные по столбцу gender не имеют значения. Оставлено без изменений.

#### Колонка income_type, debt и total_income

In [18]:
data['income_type'].sort_values().unique()

array(['безработный', 'в декрете', 'госслужащий', 'компаньон',
       'пенсионер', 'предприниматель', 'сотрудник', 'студент'],
      dtype=object)

In [19]:
data['debt'].sort_values().unique()

array([0, 1], dtype=int64)

In [20]:
data['total_income'].sort_values().unique()

array([  20667.26379327,   21205.28056622,   21367.64835649, ...,
       1726276.01433167, 2200852.2102589 , 2265604.02872274])

In [21]:
data['total_income'].describe()

count    2.152500e+04
mean     1.651595e+05
std      9.786607e+04
min      2.066726e+04
25%      1.077982e+05
50%      1.450179e+05
75%      1.955436e+05
max      2.265604e+06
Name: total_income, dtype: float64

Без аномальных значений.

#### Колонка purpose - цель кредита

In [22]:
data['purpose'].sort_values().unique()

array(['автомобили', 'автомобиль', 'высшее образование',
       'дополнительное образование', 'жилье',
       'заняться высшим образованием', 'заняться образованием',
       'на покупку автомобиля', 'на покупку подержанного автомобиля',
       'на покупку своего автомобиля', 'на проведение свадьбы',
       'недвижимость', 'образование', 'операции с жильем',
       'операции с коммерческой недвижимостью',
       'операции с недвижимостью', 'операции со своей недвижимостью',
       'покупка жилой недвижимости', 'покупка жилья',
       'покупка жилья для сдачи', 'покупка жилья для семьи',
       'покупка коммерческой недвижимости', 'покупка недвижимости',
       'покупка своего жилья', 'получение высшего образования',
       'получение дополнительного образования', 'получение образования',
       'приобретение автомобиля', 'профильное образование',
       'ремонт жилью', 'свадьба', 'свой автомобиль',
       'сделка с автомобилем', 'сделка с подержанным автомобилем',
       'строительство 

Цели повторяются в разной формулировке.

<a id='3'></a>
### 3. Изменение типов данных

В колонке total_income тип данных указан как float64. Заменим на целочисленное значение.

In [23]:
data['total_income'] = data['total_income'].astype('int')
data['total_income'].sort_values().unique()

array([  20667,   21205,   21367, ..., 1726276, 2200852, 2265604])

In [24]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     21525 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      21525 non-null  int32  
 11  purpose           21525 non-null  object 
dtypes: float64(1), int32(1), int64(5), object(5)
memory usage: 1.9+ MB


<a id='4'></a>
### 4. Удаление дубликатов

In [25]:
data['education'].sort_values().unique()

array(['ВЫСШЕЕ', 'Высшее', 'НАЧАЛЬНОЕ', 'НЕОКОНЧЕННОЕ ВЫСШЕЕ',
       'Начальное', 'Неоконченное высшее', 'СРЕДНЕЕ', 'Среднее',
       'УЧЕНАЯ СТЕПЕНЬ', 'Ученая степень', 'высшее', 'начальное',
       'неоконченное высшее', 'среднее', 'ученая степень'], dtype=object)

In [26]:
data['education'] = data['education'].str.lower()
data['education'].sort_values().unique()

array(['высшее', 'начальное', 'неоконченное высшее', 'среднее',
       'ученая степень'], dtype=object)

In [27]:
data.duplicated().sum()

72

In [28]:
data = data.drop_duplicates().reset_index(drop=True)
data.duplicated().sum()


0

In [29]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21453 entries, 0 to 21452
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21453 non-null  int64  
 1   days_employed     21453 non-null  float64
 2   dob_years         21453 non-null  int64  
 3   education         21453 non-null  object 
 4   education_id      21453 non-null  int64  
 5   family_status     21453 non-null  object 
 6   family_status_id  21453 non-null  int64  
 7   gender            21453 non-null  object 
 8   income_type       21453 non-null  object 
 9   debt              21453 non-null  int64  
 10  total_income      21453 non-null  int32  
 11  purpose           21453 non-null  object 
dtypes: float64(1), int32(1), int64(5), object(5)
memory usage: 1.9+ MB


В процессе анализа по столбцу education выявлены неявные дубликаты, связанные с применением в значении разных регистров. Исправлено приведением к одному. Также обнаружены дубликаты строк в количестве 72. После удаления общее число строк уменьшилось до 21453.

<a id='5'></a>
### 5. Формирование дополнительных датафреймов словарей, декомпозиция исходного датафрейма

In [30]:
education_dict = data[['education_id', 'education']]
education_dict = education_dict.drop_duplicates().reset_index(drop=True)
education_dict

,education_id,education
0,0,высшее
1,1,среднее
2,2,неоконченное высшее
3,3,начальное
4,4,ученая степень


In [31]:
family_status_dict = data[['family_status_id', 'family_status']]
family_status_dict =family_status_dict.drop_duplicates().reset_index(drop=True)
family_status_dict 

,family_status_id,family_status
0,0,женат / замужем
1,1,гражданский брак
2,2,вдовец / вдова
3,3,в разводе
4,4,Не женат / не замужем


In [32]:
data = data[['children', 'days_employed', 'dob_years', 'education', 'family_status', 'gender', 'income_type', 'debt', 'total_income', 'purpose']]
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21453 entries, 0 to 21452
Data columns (total 10 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   children       21453 non-null  int64  
 1   days_employed  21453 non-null  float64
 2   dob_years      21453 non-null  int64  
 3   education      21453 non-null  object 
 4   family_status  21453 non-null  object 
 5   gender         21453 non-null  object 
 6   income_type    21453 non-null  object 
 7   debt           21453 non-null  int64  
 8   total_income   21453 non-null  int32  
 9   purpose        21453 non-null  object 
dtypes: float64(1), int32(1), int64(3), object(5)
memory usage: 1.6+ MB


<a id='6'></a>
### 6. Категоризация дохода

In [33]:
def category (total_income):
    if total_income <= 30000:
        return 'E'
    if total_income <= 50000:
        return 'D'
    if total_income <= 200000:
        return 'C'
    if total_income <= 1000000:
        return 'B'
    return 'A'

data['total_income_category'] = data['total_income'].apply(category)
data.head(10)

,children,days_employed,dob_years,education,family_status,gender,income_type,debt,total_income,purpose,total_income_category
0,1,8437.673028,42,высшее,женат / замужем,F,сотрудник,0,253875,покупка жилья,B
1,1,4024.803754,36,среднее,женат / замужем,F,сотрудник,0,112080,приобретение автомобиля,C
2,0,5623.422610,33,среднее,женат / замужем,M,сотрудник,0,145885,покупка жилья,C
3,3,4124.747207,32,среднее,женат / замужем,M,сотрудник,0,267628,дополнительное образование,B
4,0,340266.072047,53,среднее,гражданский брак,F,пенсионер,0,158616,сыграть свадьбу,C
5,0,926.185831,27,высшее,гражданский брак,M,компаньон,0,255763,покупка жилья,B
6,0,2879.202052,43,высшее,женат / замужем,F,компаньон,0,240525,операции с жильем,B
7,0,152.779569,50,среднее,женат / замужем,M,сотрудник,0,135823,образование,C
8,2,6929.865299,35,высшее,гражданский брак,F,сотрудник,0,95856,на проведение свадьбы,C
9,0,2188.756445,41,среднее,женат / замужем,M,сотрудник,0,144425,покупка жилья для семьи,C


<a id='7'></a>
### 7. Категоризация целей кредита

In [34]:
def purpose_category(purpose):
    if 'авто' in purpose:
        return 'операции с автомобилями'
    if 'образов' in purpose:
        return 'получение образования'
    if 'свад' in purpose:
        return 'проведение свадеб'
    return 'операции с недвижимостью'

data['purpose_category'] = data['purpose'].apply(purpose_category)
data.head(10)

,children,days_employed,dob_years,education,family_status,gender,income_type,debt,total_income,purpose,total_income_category,purpose_category
0,1,8437.673028,42,высшее,женат / замужем,F,сотрудник,0,253875,покупка жилья,B,операции с недвижимостью
1,1,4024.803754,36,среднее,женат / замужем,F,сотрудник,0,112080,приобретение автомобиля,C,операции с автомобилями
2,0,5623.422610,33,среднее,женат / замужем,M,сотрудник,0,145885,покупка жилья,C,операции с недвижимостью
3,3,4124.747207,32,среднее,женат / замужем,M,сотрудник,0,267628,дополнительное образование,B,получение образования
4,0,340266.072047,53,среднее,гражданский брак,F,пенсионер,0,158616,сыграть свадьбу,C,проведение свадеб
5,0,926.185831,27,высшее,гражданский брак,M,компаньон,0,255763,покупка жилья,B,операции с недвижимостью
6,0,2879.202052,43,высшее,женат / замужем,F,компаньон,0,240525,операции с жильем,B,операции с недвижимостью
7,0,152.779569,50,среднее,женат / замужем,M,сотрудник,0,135823,образование,C,получение образования
8,2,6929.865299,35,высшее,гражданский брак,F,сотрудник,0,95856,на проведение свадьбы,C,проведение свадеб
9,0,2188.756445,41,среднее,женат / замужем,M,сотрудник,0,144425,покупка жилья для семьи,C,операции с недвижимостью


<a id='8'></a>
### 8. Определение зависимостей

#### Определим зависимость между количеством детей и возвратом кредита в срок

In [35]:
data.pivot_table(index='children', values='debt', aggfunc=['sum', 'count', 'mean'])


,sum,count,mean
,debt,debt,debt
children,,,
0,1063,14090,0.075444
1,445,4855,0.091658
2,202,2128,0.094925
3,27,330,0.081818
4,4,41,0.097561
5,0,9,0.000000


Вывод:  Зависимость между количеством детей и возвратом кредита в срок есть - чаще кредиты берут люди без детей.
С увеличением числа детей, уменьшается количество должников, но доля должников к общему числу заёмщиков:
у "бездетных" 7,5%
с 1-м ребёнком - 9,1%
с 2-мя детьмя - 9,5%
с 3-мя детьми - 8,2%
с 4-мя детьми - 9,75%
Самые добросовестные заёмщики с числом детей - 5, больше всего просрочек с 4-мя и 2-мя детьми.

#### Определим зависимость между семейным положением и возвратом кредита в срок

In [36]:
data.pivot_table(index='family_status', values='debt', aggfunc= ['sum', 'count', 'mean'])


,sum,count,mean
,debt,debt,debt
family_status,,,
Не женат / не замужем,274,2810,0.097509
в разводе,85,1195,0.071130
вдовец / вдова,63,959,0.065693
гражданский брак,388,4150,0.093494
женат / замужем,931,12339,0.075452


Вывод:  Зависисмость между семейным положением и возвратом кредита в срок есть - чаще кредиты берут женатые/замужние, но доля должников из общего числа заёмщиков выше у не женатых/не замужних - они самые ненадежные.

Не женатые/не замужние - 9,74%  
в разводе - 7,11%  
вдовец/вдова - 6,56%  
гражданский брак - 9,28%  
женат/замужем - 7,5%  
    

#### Определим зависимость между уровнем дохода и возвратом кредита в срок

In [37]:
data.pivot_table(index='total_income_category', values='debt', aggfunc=['sum', 'count', 'mean'])


,sum,count,mean
,debt,debt,debt
total_income_category,,,
A,2,25,0.080000
B,356,5041,0.070621
C,1360,16015,0.084920
D,21,350,0.060000
E,2,22,0.090909


Вывод:  Зависисмость между уровнем дохода и возвратом кредита в срок есть - чаще за кредитом обращаются люди с доходом от 50001 до 200000, но наибольшая доля должников в общем числе заёмщиков у категории Е - с доходом до 30000.  
А - 8%  
B - 7%  
C - 8.45%  
D - 6%  
E - 9.1%  

#### Определим влияние разных целей кредита на его возврат в срок

In [38]:
data.pivot_table(index='purpose_category', values='debt', aggfunc=['sum', 'count', 'mean'])


,sum,count,mean
,debt,debt,debt
purpose_category,,,
операции с автомобилями,403,4306,0.093590
операции с недвижимостью,782,10811,0.072334
получение образования,370,4013,0.092200
проведение свадеб,186,2323,0.080069


Вывод:  Чаще за кредитом обращаются с целью приобретение недвижимости, но наибольшую долю должников из общего числа заёмщиков, те, что брали кредит на операции с автомобилями.
операции с автомобилями - 9,33%
операции с недвижимостью - 7,21%
получение образования - 9,19%
проведение свадеб - 8,04%

<a id='9'></a>
### 9.Общий вывод

**Цель исследования** - определить степень влияния семейного положения и количества детей заёмщика на факт своевременного погашения кредита, достигнута**.  

В ходе анализа были подробно изучены исходные данные, состоящие из 21525 строк и 12 колонок, по составу и содержанию:  
- Проведена предобработка: заполнены пропуски в строках, исправлены выявленные артефакты, удалены дубликаты;    
- Уникальные связки id с значениями типа образования и семейного статуса сформированы в отдельные таблицы;  
- Для удобства анализа данные о доходах и целях кредита объединены по категориям. 

Как результат - определена зависимость возврата кредита в срок от количества детей, семейного положения, уровня доходов и цели кредита: **семейное положение имеет влияние на возврат кредита в срок: не женатые/не замужние, в гражданском браке - имеют наибольшую долю "невозвратов". Заёмщики, имеющие 2-х и 4-х детей, также имеют больший процент должников.**